# RXN for Chemistry APIs: Example Notebook

In this notebook we explore the main features of `rxn4chemistry`, the python wrapper for [RXN for Chemistry](https://rxn.res.ibm.com). For a full set of features, consult the the [online documentation](https://rxn4chemistry.github.io/rxn4chemistry). Below are the tools in RXN for Chemistry. 

| Tool | Description | Ref. |
| --- | --- | --- |
| **Predict retrosynthesis** | Predict possible retrosynthetic routes given a target molecule | 1, 2 |
| **Predict product** | Predict the product of a chemical reaction given the starting materials | 3 | 
| **Predict reagents** | Predict the reagents needed to convert a given starting material to a given product | N/A | 
| **Plan a synthesis** | Plan a synthesis starting from a target molecule, a retrosynthetic route, or an experimental procedure | 4, 5 | 
| **Atom mapping** | Map atoms from starting materials to products | 6 | 
| **Text to procedure** | Translate your reaction procedures from text to exact steps to follow | 5 | 
| **Reaction digitization** | Convert images of reaction schemes to machine-readable format | N/A | 

Please refer to the references below for methodological details of the AI models. If you use RXN for Chemistry in your projects, please consider citing the relevant article(s).

1. *"Predicting retrosynthetic pathways using transformer-based models and a hyper-graph exploration strategy."* Schwaller, P.; , Petraglia, R.; Zullo, V.; Nair, V. H.; Haeuselmann, R. A.; Pisoni, R.; Bekas, C.; Iuliano, A.; Laino, T.  *Chem. Sci.*, **2020**, *11*, 3316. [[link]](https://doi.org/10.1039/C9SC05704H)
2. *"Biocatalysed synthesis planning using data-driven learning"* Probst, D.; Manica, M.; Nana Teukam, Y. G.; Castrogiovanni, A.; Paratore, F.; Laino, T. *Nat. Commun.* **2022**, *13*, 964. [[link]](https://www.nature.com/articles/s41467-022-28536-w)
3. *"Molecular Transformer: A Model for Uncertainty-Calibrated Chemical Reaction Prediction."* Schwaller, P.; Laino, T.; Gaudin, T.; Bolgar, P.; Hunter, C. A.; Bekas, C.; Lee, A. A. *ACS Cent. Sci.* **2019**, *5*, 1572. [[link]](https://doi.org/10.1021/acscentsci.9b00576)
4. *"Inferring experimental procedures from text-based representations of chemical reactions."* Vaucher, A. C.; Schwaller, P.; Geluykens, J.; Nair, V. H.; Iuliano, A.; Laino, T. *Nat. Commun.* **2021**, *12*, 2573. [[link]](https://doi.org/10.1038/s41467-021-22951-1)
5. *"Automated extraction of chemical synthesis actions from experimental procedures."* Vaucher, A. C.; Zipoli, F.; Geluykens, J.; Nair, V. H.; Schwaller, P.; Laino, T. *Nat. Commun.* **2020**, *11*, 3601. [[link]](https://doi.org/10.1038/s41467-020-17266-6)
6. *"Extraction of organic chemistry grammar from unsupervised learning of chemical reactions."* Schwaller, P.; Hoover, B; Reymond, J.-L.; Strobelt, H.; Laino, T. *Sci. Adv.* **2021**, *7*, eabe4166. [[link]](https://www.science.org/doi/10.1126/sciadv.abe4166)



## API access

Users in the free tier of RXN for Chemistry have UI access only.  For programmatic access to RXN for Chemistry, users need an Individual or Team subscription.  These plans feature full API access with no rate limitations. You can view subscription options [here](https://rxn.app.accelerate.science/rxn/user-subscription). 

## Instantiate the wrapper

Set up the wrapper using a valid API key. If you have an Individual or Teams subscription, your API key can be found on the RXN for Chemistry [profile page](https://rxn.res.ibm.com/rxn/user/profile). 



In [ ]:
from rxn4chemistry import RXN4ChemistryWrapper

api_key = "YOUR_API_KEY"
rxn = RXN4ChemistryWrapper(api_key=api_key)

## Projects

Results from the four tools below can be saved to a project.  Projects help to organize analyses and can be shared with colleagues.

- Predict retrosynthesis
- Predict product
- Predict reagents
- Plan a synthesis

### For on-premise installations (uncommon)

You can refer to a custom on-premise installation via an environment variable:

```console
export RXN4CHEMISTRY_BASE_URL="https://some.other.rxn.server"
```

Or by setting a different host in your python code:

In [ ]:
# uncomment the line below to set a different host
# rxn.set_base_url('https://some.other.rxn.server')

### Create a new project

To create a project, run the ```create_project``` function.

This step can be skipped if you want to use a project that you have created previously.

In [ ]:
rxn.create_project("rxn4chemistry_tour")
rxn.set_project(rxn.project_id)
print(f"The project ID is {rxn.project_id}")

### Choose an existing project

If you have already created the project, you can work within it by running ```set_project```.

In [ ]:
rxn.set_project("YOUR_PROJECT_ID")

## Predict retrosynthesis

### Predict possible retrosynthetic routes given a target molecule

The predict retrosynthesis tool supports both automatic and user-guided retrosynthesis.  We recommend that user-guided retrosynthesis be done within the RXN for Chemistry site since the GUI workflow is more user-friendly.  The automatic retrosynthesis option is shown below.

To predict a retrosynthesis using default parameters, simply define a molecule in SMILES format and pass it as an argument to the ```predict_automatic_retrosynthesis``` function.

In [ ]:
smiles = 'CC(=O)NC1=CC=C(Br)C=C1'
predict_automatic_retrosynthesis_response = rxn.predict_automatic_retrosynthesis(product=smiles)

Check on the status of the retrosynthesis prediction. 
- 'NEW': Job is still running.
- 'SUCCESS': Job is complete.

Rerun the cell below until 'SUCCESS' is returned.

In [ ]:
predict_automatic_retrosynthesis_results = rxn.get_predict_automatic_retrosynthesis_results(
    predict_automatic_retrosynthesis_response['prediction_id']
)
predict_automatic_retrosynthesis_results['status']

Upon 'SUCCESS' we can assess the predicted retrosynthetic paths. 

But first we define a function ```collect_reactions_from_retrosynthesis``` to parse the results of the retrosynthesis prediction.

In [ ]:
from typing import Dict, List
from IPython.display import display
from rdkit import Chem
from rdkit.Chem import AllChem

# To parse results from the predict retrosynthesis tool
def collect_reactions_from_retrosynthesis(tree: Dict) -> List[str]:
    reactions = []
    if 'children' in tree and len(tree['children']):
        reactions.append(
            AllChem.ReactionFromSmarts('{}>>{}'.format(
                '.'.join([node['smiles'] for node in tree['children']]),
                tree['smiles']
            ), useSmiles=True)
        )
    for node in tree['children']:
        reactions.extend(collect_reactions_from_retrosynthesis(node))
    return reactions

We use this helper function to display the different retrosynthesis routes produced by the tool.

In [ ]:
for index, tree in enumerate(predict_automatic_retrosynthesis_results['retrosynthetic_paths']):
    print('Showing path {} with confidence {}:'.format(index, tree['confidence']))
    for reaction in collect_reactions_from_retrosynthesis(tree):
        display(Chem.Draw.ReactionToImage(reaction))

## Predict product

### Predict the product of a chemical reaction given the starting materials

RXN for Chemistry uses a forward reaction prediction model based on molecular transformers.  

To run a forward reaction prediction, use the ```predict_reaction``` function.  Pass the starting materials in SMILES format, appended with a `.` character as shown below.

In [ ]:
predict_reaction_response = rxn.predict_reaction(
    'BrBr.c1ccc2cc3ccccc3cc2c1'
)

Rerun the cell below until SUCCESS is returned.

In [ ]:
predict_reaction_results = rxn.get_predict_reaction_results(
    predict_reaction_response['prediction_id']
)
print(predict_reaction_results["response"]["payload"]["status"])

Define a helper function to parse the output.

In [ ]:
def get_reaction_from_smiles(reaction_smiles: str) -> Chem.rdChemReactions.ChemicalReaction:
    return AllChem.ReactionFromSmarts(reaction_smiles, useSmiles=True)

Use the helper function to show the predicted product.

In [ ]:
get_reaction_from_smiles(predict_reaction_results['response']['payload']['attempts'][0]['smiles'])

It is also possible to run forward reaction predictions in batches to use the service in a high-throughput fashion.  Note that this will not store the information in any project.

In [ ]:
predict_rection_batch_response = rxn.predict_reaction_batch(
    precursors_list=['BrBr.c1ccc2cc3ccccc3cc2c1', 'Cl.c1ccc2cc3ccccc3cc2c1']
)

Rerun the cell below until a dictionary is returned.

In [ ]:
result = rxn.get_predict_reaction_batch_results(
    predict_rection_batch_response['task_id']
)
print(result)

Then we can dispay the prediction results for each of the reactions in the batch.

In [ ]:
for reaction_prediction in result["predictions"]:
    print(f'Confidence: {reaction_prediction["confidence"]}')
    display(get_reaction_from_smiles(reaction_prediction['smiles']))

It is also possible to predict the top N products for each reaction (in batch). 

In [ ]:
response = rxn.predict_reaction_batch_topn(
    precursors_lists=[
        ["BrBr", "c1ccc2cc3ccccc3cc2c1"],
        ["BrBr", "c1ccc2cc3ccccc3cc2c1CCO"],
        ["Cl", "CCC(=O)NCCC", "O"],
    ],
    topn=5,
)

Rerun the cell below until SUCCESS is returned.

In [ ]:
result = rxn.get_predict_reaction_batch_topn_results(
    response["task_id"]
)
try:
    result["predictions"]
    print("SUCCESS")
except:
    print("Prediction not ready, try again")

Then we list the predicted products, alongside their confidences, for each of the three reactions.

In [ ]:
for i, reaction_predictions in enumerate(result['predictions'], 1):
    print(f'Outcomes for reaction no {i}:')
    for j, prediction in enumerate(reaction_predictions["results"], 1):
        product_smiles = ".".join(prediction["smiles"])
        confidence = prediction["confidence"]
        print(f'  Product(s) {j}: {product_smiles}, with confidence {confidence}')

**NOTE:** The results for batch predictions are only stored temporarily in our databases, so we strongly recommend saving them elsewhere.

## Predict reagents

### Predict the reagents needed to convert a given starting material to a given product

This tool predicts which reagents would allow you to convert your starting material to the desired product.

In [ ]:
starting_material_smiles = 'C1=C(C=O)C=C(Br)C=C1'
product_smiles = 'C1C=CC(Br)=CC=1/C=C/C(O)=O'
response = rxn.predict_reagents(
    starting_material_smiles,
    product_smiles
)
print(response["prediction_id"])

Rerun the cell below until SUCCESS is returned.

In [ ]:
result = rxn.get_predict_reagents_results(response["prediction_id"])
print(result["response"]["payload"]["status"])

Display the overall predicted reaction.

In [ ]:
for item in result["response"]["payload"]["sequences"]:
    print(f'Confidence: {item["confidence"]}')
    display(AllChem.ReactionFromSmarts(item["smiles"]))

## Plan a synthesis

### Plan a synthesis starting from a target molecule, a retrosynthetic route, or an experimental procedure in text format

**NOTE:** We recommend this tool be used within the RXN for Chemistry site instead of via API (as shown here), since the GUI workflow is more user-friendly.

We will convert our retrosynthesis prediction from above into a synthesis plan. The cell below may take a short while to run.

In [ ]:
create_synthesis_from_sequence_response = rxn.create_synthesis_from_sequence(
    sequence_id=predict_automatic_retrosynthesis_results['retrosynthetic_paths'][1]['sequenceId']
)
print(f'Identifier for the synthesis: {create_synthesis_from_sequence_response["synthesis_id"]}')

Get the actions predicted by the AI model.

In [ ]:
synthesis_id = create_synthesis_from_sequence_response['synthesis_id']
node_id = rxn.get_node_ids(synthesis_id=synthesis_id)[-1]

Print the actions.

In [ ]:
actions_and_product = rxn.get_reaction_settings(synthesis_id=synthesis_id, node_id=node_id)
print(actions_and_product)

In [ ]:
import json

node_actions, product = actions_and_product['actions'], actions_and_product['product']

for index, action in enumerate(node_actions, 1):
    print(f'Action {index}:\n{json.dumps(action, indent=4)}\n')

The adding acetyl chloride action needs to be changed to not be dropwise since solids are added in pins. We also remove the purify action since it is not currently supported by commonly used robotic hardware.


In [ ]:
# update the the action so the solid is not added dropwise
node_actions[3]['content']['dropwise']['value'] = False

# remove the purify action 
node_actions.pop(11)

# update the node actions
rxn.update_reaction_settings(synthesis_id=synthesis_id, node_id=node_id, actions=node_actions, product=product)

## Atom mapping

### Map atoms from starting materials to product

The ```predict_reaction_properties``` function will perform atom mapping as shown below.

In [ ]:
reaction_prop = rxn.predict_reaction_properties(
    ['CC(=O)[OH]>>CC(=O)OCC']
)
print(reaction_prop['response']['payload']['content'][0]['value'])

## Text to procedure

### Translate your reaction procedures from text to exact steps to follow

Extract machine-readable actions from text descriptions of chemical procedures in paragraph format.  To extract the actions from a recipe, pass the description as a string to the `paragraph_to_actions` function.

In [ ]:
actions_from_procedure_results = rxn.paragraph_to_actions(
    'To a stirred solution of '
    '7-(difluoromethylsulfonyl)-4-fluoro-indan-1-one (110 mg, '
    '0.42 mmol) in methanol (4 mL) was added sodium borohydride '
    '(24 mg, 0.62 mmol). The reaction mixture was stirred at '
    'ambient temperature for 1 hour.'
)

The output is a standardized procedure, which can be a useful starting point for passing information to a robotic platform.

In [ ]:
for index, action in enumerate(actions_from_procedure_results['actions'], 1):
    print(f'{index}. {action}')

## Reaction digitization

### Convert images of reaction schemes to machine-readable format

**NOTE:** We recommend this tool be used within the RXN for Chemistry site instead of via API (as shown here), since the GUI workflow is more user-friendly.

In [ ]:
# Upload the picture and get back the file id
# Use full path and ensure the path is correct
try:
    response = rxn.upload_file('./reaction_scheme.png')
    file_id = response['response']['payload']['id']
    print(file_id)
except FileNotFoundError:
    print("File path provided does not exist")

Use the file_id to start the digitization process. This may take some time.

In [ ]:
result = rxn.digitize_reaction(file_id)
result['response']['payload']['reactions']

Display the results in a human-readable way.

In [ ]:
for item in result["response"]["payload"]["reactions"]:
    print(item["reactant"]["content"])
    print(f"   --- {item['text']['content']} --->")
    print(item["product"]["content"])
    print()